# Multi-Omics Integration for Personalized Medicine Using MOFA

In this notebook, you'll explore a multi-omic dataset of Chronic Lymphocytic Leukemia patients using Multi-Omics Factor Analysis (MOFA) with the MOFA-FLEX package. MOFA-FLEX is a factor model framework that combines functionalities from various previously published methods, including MOFA.

## Prerequisites

Before getting started, ensure that you're familiar with the Python packages used in this tutorial. Below you can find links for short introductions.
- [NumPy](https://numpy.org/doc/2.2/user/quickstart.html)
- [Pandas](https://pandas.pydata.org/docs/user_guide/10min.html)
- [AnnData](https://anndata.readthedocs.io/en/stable/tutorials/notebooks/getting-started.html)
- [ScanPy](https://scanpy.readthedocs.io/en/stable/tutorials/index.html)
- [Plotnine](https://plotnine.org/guide/overview.html) (similar to ggplot2 in R)

In [ ]:
import os
import numpy as np
import pandas as pd
import anndata as ad
import scanpy as sc
import plotnine as p9
import gseapy as gp
from scipy.stats import pearsonr
from sklearn.neighbors import KNeighborsClassifier

import mofaflex as mf

## Loading and Examining the Data

The data consists of measurements of four modalities:
- In vitro drug response
- Methylation
- mRNA
- Somatic mutations

The data has already been prepared and is available as `.h5ad` files, which you can load using AnnData's `read_h5ad` function.

In [ ]:
adata_drugs = ad.read_h5ad("data/Drugs.h5ad")
adata_methylation = ad.read_h5ad("data/Methylation.h5ad")
adata_mrna = ad.read_h5ad("data/mRNA.h5ad")
adata_mutations = ad.read_h5ad("data/Mutations.h5ad")

### Exercise
1. Examine the AnnData objects for the different modalities. Do they share the same observations or features?
2. Get a feeling for the values in the individual modalities. It's always a good idea to plot their distribution. Why? When do you have to apply further preprocessing before continuing with the actual analysis?

In [ ]:
# your code

## Training a MOFA-FLEX Model

MOFA-FLEX assumes input data that is structured into observation groups ("groups") and variable groups ("views"), as shown in [Figure 1a of the MOFA+ paper](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-020-02015-1/figures/1). Views can e.g. be different measurement modalities like RNA and DNA. This structure is represented as a nested Python dictionary of AnnData objects. The outer level represents the groups, while the inner level represents the views:

`input_data = {"group_1": {"view_1": adata_g1_v1, "view_2": adata_g1_v2}, "group_2": {"view_1": adata_g2_v1, "view_2": adata_g2_v2}}`

### Exercise:
Prepare the CLL input data for MOFA-FLEX. Note that there is only one observation group.

In [ ]:
# your code

MOFA-FLEX can be used in many different configurations and needs to be configured to work well with the specifics of a particular data set. A new MOFA-FLEX model is instantiated and trained using the `mf.MOFAFLEX` class. To configure the model, Options classes are used: `mf.DataOptions`, `mf.ModelOptions`, `mf.TrainingOptions` and `mf.SmoothOptions`. Read more about the different possibilities in the [documentation](https://mofaflex.readthedocs.io/latest/api/index.html).

### Exercise
Train a MOFA-FLEX model on the CLL data. You'll want to use this configuration (everything else should be default values):
- 10 factors
- Horseshoe weight prior
- Normal likelihood for all views
- Training for no more than 10 000 epochs
- Learning rate of 0.05
- Early stopping patience of 1000 epochs
- Save the model with your favourite file name

You'll see that you also get a nice overview of the data structure before training starts.
While the model is training, why don't you have a look into the [original MOFA paper](https://www.embopress.org/doi/full/10.15252/msb.20178124)? Look for the supplementary methods if you're interested in learning more about the statistics involved.

In [ ]:
# your code

One of the first things you'll want to look at after model training has finished is how meaningful the factors are that were found. A good measure for this is the explained variance per factor. It tells you how much of the observed variance in a given view is explained by an individual factor. MOFA-FLEX can create a plot of this quantity with the `mf.pl.variance_explained` function.

Other important checks are the convergence of the training loss curve and the factor correlation. The factor correlation is a cross correlation matrix of individual factors, showing the user how similar the factors are to each other. 

### Exercise
1. Make a variance explained plot and explain what you're seeing!
2. Can you find the relevant functions in `mf.pl` for plotting the training loss curve and the factor correlation? Do the plots look good?

In [ ]:
# your code

## Choosing the right number of factors
The number of factors used in a MOFA-FLEX model is a user choice which can have some influence on the outcome.

### Exercise
1. Train a MOFA-FLEX model with a smaller number of factors and re-do the analysis from above. Make sure to give it another name (and save to another file) to have access to both models afterwards. Do you see any difference?
2. What could be consequences of choosing too many or too few factors?

In [ ]:
# your code

## Examining the factor weights and scores of the trained model

Now we'll want to have a closer look at the two main results of a MOFA-FLEX analysis:
1. Factor weights (matrix of shape `(n_observations, n_factors)`), representing the activity of individual factors in each observation
2. Factor scores (matrix of shape `(n_variables, n_factors)`), linking the low-dimensional factors to the high-dimensional original variables (e.g. genes)

We're sometimes a bit sloppy with the language and refer to the factors scores simply as "factors", and to the factor weights as "weights".

### Exercise
1. The factor scores and weights are obtained from the MOFA-FLEX model instance using the `.get_factors()` and `.get_weights()` methods, respectively. Try this out and validate the dimensions of the two matrices. Why do you get dictionaries from these methods and how are they structured?
2. To get a very quick overview, make heatmaps of the two matrices. There is no inbuilt method for this (use plotnine or your favourite plotting library).

In [ ]:
# your code

### Exercise

You have now trained two MOFA-FLEX models, one with 10 factors and one with a smaller number. How similar are inidividual factors (in terms of their factor scores) between those two? To find this out, compute the Pearson correlation coefficient for each possible combination of factors across models to create a correlation matrix (shape `10 x smaller number of factors`). What do you see and does this match your expectations?

In [ ]:
# your code

### Exercise

1. Let's now look at individual factors. You can use the `mf.pl.factors_scatter` method to make a scatterplot of the factor scores of one factor vs another. Do this for the two factors in the first model (the one with 10 factors). Do you see any structure?
2. If there are observation covariates, it makes sense to color the scatterplot according to these to explore whether the observed structure corresponds to any of the covariates. This is done using the `color` argument of the plotting function, which can be given the column name of the respective covariate in the `.obs` DataFrame of the original input data. Try this for the different available covariates and see whether you can find something!
3. Can you find a factor that separates patients by their trisomy12 status?

In [ ]:
# your code

### Exercise

You have probably seen that not all patients have known IGHV status (they just have a NaN in the corresponding column). However, you just found a factor that separates patients by their IGHV status! How about predicting IGHV status for the missing patients from the factor scores of this factor? Use a simple classification algorithm from scikit-learn, such as `KNeighborsClassifier`. This algorithm takes as input the factor scores for the relevant factor and decides for each of the missing patients based on a majority vote of the surrounding patients (surrounding means closest factor scores). Hint: Use only the patients with known IGHV status as a training set, and then predict IGHV status on the full data set.

In [ ]:
# your code

Now that you found which factors stratify the patients into several groups, you should have a closer look at the corresponding factor weights. The weights describe how each original variable (gene, methylation site, ...) is associated with an individual factor. Also note that both the factor scores and weights can be positive or negative. Switching the signs of both the scores and the weights for a factor will not make a difference because the $-1 * -1 = 1$. But in general, the sign is important because it tells us about the direction of an effect. For example, if the sign of the factor weights for gene A is positive, and for gene B is negative, an increased activity (as measured by the factor score) of this factor in a given sample has opposite effects on the values of those genes in the data.

## Exercise
Use the `mf.pl.top_weights` function to plot the highest (absolute) weights of the factor that you found to separate patients by IGHV status. Plot the weights for the different modalities (views). What can you conclude about e.g. a drug with a high weight in the drugs view? How is it related to IGHV status and can you make predictions about drug efficacy from this? What role does the sign of the weight play?

In [ ]:
# your code

Finally, we want to try assigning biological meaning to the factors that we found. This could e.g. be a pathway that comprises a number of the original data variables. This is done by looking at the factor weights. For each factor, we can sort the weights by their absolute value and do a gene set enrichment analysis with the top hits.

### Exercise
1. Get the factor weights for the mRNA view and sort the genes by their absolute value.
2. Use GSEAPY (use the [prerank module](https://gseapy.readthedocs.io/en/latest/gseapy_example.html#Prerank-example)) to perform an enrichment analysis of gene sets in the Reactome data base. Inspect the results and try to make biological sense of them.

In [ ]:
# your code